In [0]:
%fs ls

path,name,size,modificationTime
dbfs:/FileStore/,FileStore/,0,0
dbfs:/databricks-datasets/,databricks-datasets/,0,0
dbfs:/databricks-results/,databricks-results/,0,0
dbfs:/user/,user/,0,0


In [0]:
%fs ls dbfs:/FileStore/

path,name,size,modificationTime
dbfs:/FileStore/tables/,tables/,0,0


In [0]:
# Import necessary libraries
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("coffee_Shop_sales_analysis") \
    .getOrCreate()


df = spark.read.csv("dbfs:/FileStore/tables/", header=True, inferSchema=True)

df.show(5)


+--------------+----------------+-------------------+---------------+--------+---------------+----------+----------+------------------+--------------------+--------------------+
|transaction_id|transaction_date|   transaction_time|transaction_qty|store_id| store_location|product_id|unit_price|  product_category|        product_type|      product_detail|
+--------------+----------------+-------------------+---------------+--------+---------------+----------+----------+------------------+--------------------+--------------------+
|             1|      2023-01-01|2024-08-14 07:06:11|              2|       5|Lower Manhattan|        32|       3.0|            Coffee|Gourmet brewed co...|         Ethiopia Rg|
|             2|      2023-01-01|2024-08-14 07:08:56|              2|       5|Lower Manhattan|        57|       3.1|               Tea|     Brewed Chai tea|Spicy Eye Opener ...|
|             3|      2023-01-01|2024-08-14 07:14:04|              2|       5|Lower Manhattan|        59|     

In [0]:
df.take(5)

Out[2]: [Row(transaction_id=1, transaction_date=datetime.date(2023, 1, 1), transaction_time=datetime.datetime(2024, 8, 14, 7, 6, 11), transaction_qty=2, store_id=5, store_location='Lower Manhattan', product_id=32, unit_price=3.0, product_category='Coffee', product_type='Gourmet brewed coffee', product_detail='Ethiopia Rg'),
 Row(transaction_id=2, transaction_date=datetime.date(2023, 1, 1), transaction_time=datetime.datetime(2024, 8, 14, 7, 8, 56), transaction_qty=2, store_id=5, store_location='Lower Manhattan', product_id=57, unit_price=3.1, product_category='Tea', product_type='Brewed Chai tea', product_detail='Spicy Eye Opener Chai Lg'),
 Row(transaction_id=3, transaction_date=datetime.date(2023, 1, 1), transaction_time=datetime.datetime(2024, 8, 14, 7, 14, 4), transaction_qty=2, store_id=5, store_location='Lower Manhattan', product_id=59, unit_price=4.5, product_category='Drinking Chocolate', product_type='Hot chocolate', product_detail='Dark chocolate Lg'),
 Row(transaction_id=4, t

In [0]:
# Number of rows
num_rows = df.count()

# Number of columns
num_columns = len(df.columns)

# Print the shape of the DataFrame
print(f"Shape: ({num_rows}, {num_columns})")

Shape: (447402, 11)


In [0]:
# Print the schema of the DataFrame (data types of each column)
df.printSchema()

# Show summary statistics (e.g., count, mean, stddev, min, max) for numeric columns
df.describe().show()

root
 |-- transaction_id: integer (nullable = true)
 |-- transaction_date: date (nullable = true)
 |-- transaction_time: timestamp (nullable = true)
 |-- transaction_qty: integer (nullable = true)
 |-- store_id: integer (nullable = true)
 |-- store_location: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- unit_price: double (nullable = true)
 |-- product_category: string (nullable = true)
 |-- product_type: string (nullable = true)
 |-- product_detail: string (nullable = true)

+-------+-----------------+------------------+-----------------+---------------+------------------+------------------+----------------+----------------+--------------------+
|summary|   transaction_id|   transaction_qty|         store_id| store_location|        product_id|        unit_price|product_category|    product_type|      product_detail|
+-------+-----------------+------------------+-----------------+---------------+------------------+------------------+----------------+---------

#Handling Missing and Duplicate Values

In [0]:
# Missing values
from pyspark.sql.functions import col

df.filter(
    col("transaction_id").isNull()| col("transaction_date").isNull() | col("transaction_time").isNull()| col("transaction_qty").isNull()| col("store_id").isNull()| col("store_location").isNull()| col("product_id").isNull()| col("unit_price").isNull()  | col("product_category").isNull()| col("product_type").isNull()| col("product_detail").isNull()
).show()

+--------------+----------------+-------------------+---------------+--------+---------------+----------+----------+----------------+-----------------+--------------------+
|transaction_id|transaction_date|   transaction_time|transaction_qty|store_id| store_location|product_id|unit_price|product_category|     product_type|      product_detail|
+--------------+----------------+-------------------+---------------+--------+---------------+----------+----------+----------------+-----------------+--------------------+
|            13|            null|2024-08-14 07:45:51|              1|       5|Lower Manhattan|        51|       3.0|             Tea| Brewed Black tea|        Earl Grey Lg|
|            14|      2023-01-01|2024-08-14 07:48:19|              1|       5|           null|        57|       3.1|             Tea|  Brewed Chai tea|Spicy Eye Opener ...|
|            15|      2023-01-01|2024-08-14 07:52:36|              2|       5|Lower Manhattan|        87|      null|            null| B

In [0]:
# Alternative code for checking null values
from pyspark.sql.functions import col
from functools import reduce

# Build the condition dynamically
null_condition = reduce(lambda x, y: x | y, [col(c).isNull() for c in df.columns])

# Filter the DataFrame based on the condition
df.filter(null_condition).count()

Out[6]: 18

In [0]:
df.filter(null_condition).show()

+--------------+----------------+-------------------+---------------+--------+---------------+----------+----------+----------------+-----------------+--------------------+
|transaction_id|transaction_date|   transaction_time|transaction_qty|store_id| store_location|product_id|unit_price|product_category|     product_type|      product_detail|
+--------------+----------------+-------------------+---------------+--------+---------------+----------+----------+----------------+-----------------+--------------------+
|            13|            null|2024-08-14 07:45:51|              1|       5|Lower Manhattan|        51|       3.0|             Tea| Brewed Black tea|        Earl Grey Lg|
|            14|      2023-01-01|2024-08-14 07:48:19|              1|       5|           null|        57|       3.1|             Tea|  Brewed Chai tea|Spicy Eye Opener ...|
|            15|      2023-01-01|2024-08-14 07:52:36|              2|       5|Lower Manhattan|        87|      null|            null| B

In [0]:
df= df.dropna()

In [0]:
# Missing values
from pyspark.sql.functions import col

df.filter(
    col("transaction_id").isNull()| col("transaction_date").isNull() | col("transaction_time").isNull()| col("transaction_qty").isNull()| col("store_id").isNull()| col("store_location").isNull()| col("product_id").isNull()| col("unit_price").isNull()  | col("product_category").isNull()| col("product_type").isNull()| col("product_detail").isNull()
).show()

+--------------+----------------+----------------+---------------+--------+--------------+----------+----------+----------------+------------+--------------+
|transaction_id|transaction_date|transaction_time|transaction_qty|store_id|store_location|product_id|unit_price|product_category|product_type|product_detail|
+--------------+----------------+----------------+---------------+--------+--------------+----------+----------+----------------+------------+--------------+
+--------------+----------------+----------------+---------------+--------+--------------+----------+----------+----------------+------------+--------------+



# To Check the Duplicate values

In [0]:
from pyspark.sql.functions import count
# Step 1: Group by all columns and count occurrences
duplicates = df.groupBy(df.columns).count()

# Step 2: Filter rows where count > 1 (these are duplicates)
duplicate_rows = duplicates.filter("count > 1")

# Step 3: Show duplicate rows (optional, to see which rows are duplicated)
duplicate_rows.show()

# Step 4: Count the number of duplicate rows
duplicate_count = duplicate_rows.selectExpr("sum(count - 1) as TotalDuplicates").collect()[0]["TotalDuplicates"]

print(f"Number of duplicate rows: {duplicate_count}")

+--------------+----------------+-------------------+---------------+--------+---------------+----------+----------+------------------+--------------------+--------------------+-----+
|transaction_id|transaction_date|   transaction_time|transaction_qty|store_id| store_location|product_id|unit_price|  product_category|        product_type|      product_detail|count|
+--------------+----------------+-------------------+---------------+--------+---------------+----------+----------+------------------+--------------------+--------------------+-----+
|            60|      2023-01-01|2024-08-14 09:29:56|              1|       8| Hell's Kitchen|        74|       3.5|            Bakery|            Biscotti|     Ginger Biscotti|    3|
|           550|      2023-01-01|2024-08-14 19:58:52|              1|       3|        Astoria|        39|      4.25|            Coffee|    Barista Espresso|            Latte Rg|    3|
|           867|      2023-01-02|2024-08-14 14:59:08|              1|       8| H

In [0]:
df = df.dropDuplicates()

In [0]:
# Step 1: Group by all columns and count occurrences
duplicates = df.groupBy(df.columns).count()

# Step 2: Filter rows where count > 1 (these are duplicates)
duplicate_rows = duplicates.filter("count > 1")

# Step 3: Show duplicate rows (optional, to see which rows are duplicated)
duplicate_rows.show()

# Step 4: Count the number of duplicate rows
duplicate_count = duplicate_rows.selectExpr("sum(count - 1) as TotalDuplicates").collect()[0]["TotalDuplicates"]

print(f"Number of duplicate rows: {duplicate_count}")

+--------------+----------------+----------------+---------------+--------+--------------+----------+----------+----------------+------------+--------------+-----+
|transaction_id|transaction_date|transaction_time|transaction_qty|store_id|store_location|product_id|unit_price|product_category|product_type|product_detail|count|
+--------------+----------------+----------------+---------------+--------+--------------+----------+----------+----------------+------------+--------------+-----+
+--------------+----------------+----------------+---------------+--------+--------------+----------+----------+----------------+------------+--------------+-----+

Number of duplicate rows: None


In [0]:
df.describe().show()

+-------+-----------------+------------------+-----------------+---------------+------------------+------------------+----------------+----------------+--------------------+
|summary|   transaction_id|   transaction_qty|         store_id| store_location|        product_id|        unit_price|product_category|    product_type|      product_detail|
+-------+-----------------+------------------+-----------------+---------------+------------------+------------------+----------------+----------------+--------------------+
|  count|           149116|            149116|           149116|         149116|            149116|            149116|          149116|          149116|              149116|
|   mean|74737.37187156308| 1.438276241315486|5.342062555326055|           null| 47.91860699053086|3.3822194130743495|            null|            null|                null|
| stddev|43153.60001591792|0.5425087647372187|2.074240591571703|           null|17.930020247641586|2.6587226166627147|            

# Explore the frequency of different categories,type and detail

In [0]:
from pyspark.sql import functions as F

# List of tuples with column name and count of unique values
unique_counts = [(c, df.select(F.col(c)).distinct().count()) for c in df.columns]

# Create a DataFrame from the list of tuples
columnValue = spark.createDataFrame(unique_counts, ["Column_name", "Unique_values"])
columnValue.show()

+----------------+-------------+
|     Column_name|Unique_values|
+----------------+-------------+
|  transaction_id|       149116|
|transaction_date|          181|
|transaction_time|        25762|
| transaction_qty|            6|
|        store_id|            3|
|  store_location|            3|
|      product_id|           80|
|      unit_price|           41|
|product_category|            9|
|    product_type|           29|
|  product_detail|           80|
+----------------+-------------+



In [0]:
# Explore the frequency of different categories
df.groupBy("product_category").count().orderBy("count", ascending=False).show()

+------------------+-----+
|  product_category|count|
+------------------+-----+
|            Coffee|58416|
|               Tea|45449|
|            Bakery|22796|
|Drinking Chocolate|11468|
|          Flavours| 6790|
|      Coffee beans| 1753|
|         Loose Tea| 1210|
|           Branded|  747|
|Packaged Chocolate|  487|
+------------------+-----+



In [0]:
df.groupBy("product_type").count().orderBy("count", ascending=False).show()

+--------------------+-----+
|        product_type|count|
+--------------------+-----+
|     Brewed Chai tea|17183|
|Gourmet brewed co...|16912|
|    Barista Espresso|16403|
|       Hot chocolate|11468|
|    Brewed Black tea|11350|
|   Brewed herbal tea|11245|
|               Scone|10173|
|Organic brewed co...| 8489|
|         Drip coffee| 8477|
|Premium brewed co...| 8135|
|              Pastry| 6912|
|            Biscotti| 5711|
|    Brewed Green tea| 5671|
|       Regular syrup| 4979|
|    Sugar free syrup| 1811|
|          Housewares|  526|
|            Chai tea|  443|
|       Organic Beans|  415|
|       Gourmet Beans|  366|
|       Premium Beans|  336|
+--------------------+-----+
only showing top 20 rows



# Perform Data Transformation

In [0]:
# Calculate 'Total Sales'
df = df.withColumn("Total_sales", col("unit_price") * col("transaction_qty"))

df.show()

+--------------+----------------+-------------------+---------------+--------+---------------+----------+----------+------------------+--------------------+--------------------+-----------+
|transaction_id|transaction_date|   transaction_time|transaction_qty|store_id| store_location|product_id|unit_price|  product_category|        product_type|      product_detail|Total_sales|
+--------------+----------------+-------------------+---------------+--------+---------------+----------+----------+------------------+--------------------+--------------------+-----------+
|            15|      2023-01-01|2024-08-14 07:52:36|              2|       5|Lower Manhattan|        87|       3.0|            Coffee|    Barista Espresso|Ouro Brasileiro shot|        6.0|
|            18|      2023-01-01|2024-08-14 08:00:18|              1|       8| Hell's Kitchen|        42|       2.5|               Tea|   Brewed herbal tea|      Lemon Grass Rg|        2.5|
|             2|      2023-01-01|2024-08-14 07:08:

In [0]:
from pyspark.sql import functions as F

# Assuming 'transaction_date' is in string format, convert it to DateType
df = df.withColumn("transaction_date", F.to_date(F.col("transaction_date"), "dd/MM/yyyy"))
df = df.withColumn("transaction_time", F.col("transaction_time").cast("timestamp"))
df.show()

+--------------+----------------+-------------------+---------------+--------+---------------+----------+----------+------------------+--------------------+--------------------+-----------+
|transaction_id|transaction_date|   transaction_time|transaction_qty|store_id| store_location|product_id|unit_price|  product_category|        product_type|      product_detail|Total_sales|
+--------------+----------------+-------------------+---------------+--------+---------------+----------+----------+------------------+--------------------+--------------------+-----------+
|            15|      2023-01-01|2024-08-14 07:52:36|              2|       5|Lower Manhattan|        87|       3.0|            Coffee|    Barista Espresso|Ouro Brasileiro shot|        6.0|
|            18|      2023-01-01|2024-08-14 08:00:18|              1|       8| Hell's Kitchen|        42|       2.5|               Tea|   Brewed herbal tea|      Lemon Grass Rg|        2.5|
|             2|      2023-01-01|2024-08-14 07:08:

In [0]:
df.printSchema()

root
 |-- transaction_id: integer (nullable = true)
 |-- transaction_date: date (nullable = true)
 |-- transaction_time: timestamp (nullable = true)
 |-- transaction_qty: integer (nullable = true)
 |-- store_id: integer (nullable = true)
 |-- store_location: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- unit_price: double (nullable = true)
 |-- product_category: string (nullable = true)
 |-- product_type: string (nullable = true)
 |-- product_detail: string (nullable = true)
 |-- Total_sales: double (nullable = true)



In [0]:
# Add a new column 'year' extracted from 'transaction_date'
df = df.withColumn("year", F.year(F.col("transaction_date")))

# Add a new column 'month' extracted from 'transaction_date'
df = df.withColumn("month", F.date_format(F.col("transaction_date"), "MMMM"))

# Add a new column 'day' extracted from 'transaction_date'
df = df.withColumn("day", F.date_format(F.col("transaction_date"), "EEEE"))

df.show()

+--------------+----------------+-------------------+---------------+--------+---------------+----------+----------+------------------+--------------------+--------------------+-----------+----+-------+---------+
|transaction_id|transaction_date|   transaction_time|transaction_qty|store_id| store_location|product_id|unit_price|  product_category|        product_type|      product_detail|Total_sales|year|  month|      day|
+--------------+----------------+-------------------+---------------+--------+---------------+----------+----------+------------------+--------------------+--------------------+-----------+----+-------+---------+
|           662|      2023-01-02|2024-08-14 11:14:15|              2|       3|        Astoria|        32|       3.0|            Coffee|Gourmet brewed co...|         Ethiopia Rg|        6.0|2023|January|   Monday|
|           729|      2023-01-02|2024-08-14 12:26:13|              2|       5|Lower Manhattan|        25|       2.2|            Coffee|Organic brewe

In [0]:
# Get unique values in the 'year' ,'month','day'column
unique_year = df.select("year").distinct().show()
unique_month = df.select("month").distinct().show()
unique_day = df.select("day").distinct().show()
unique_product_category = df.select("product_category").distinct().show()
unique_store_location = df.select("store_location").distinct().show()

+----+
|year|
+----+
|2023|
+----+

+--------+
|   month|
+--------+
|February|
| January|
|   March|
|   April|
|     May|
|    June|
+--------+

+---------+
|      day|
+---------+
|Wednesday|
|  Tuesday|
|   Friday|
| Thursday|
| Saturday|
|   Monday|
|   Sunday|
+---------+

+------------------+
|  product_category|
+------------------+
|            Bakery|
|         Loose Tea|
|Drinking Chocolate|
|               Tea|
|           Branded|
|            Coffee|
|      Coffee beans|
|Packaged Chocolate|
|          Flavours|
+------------------+

+---------------+
| store_location|
+---------------+
|Lower Manhattan|
| Hell's Kitchen|
|        Astoria|
+---------------+



In [0]:
# Add a new column 'hour' extracted from 'transaction_time'
df = df.withColumn("hour", F.hour(F.col("transaction_time")))

# Add a new column 'hour' extracted from 'transaction_time'
df = df.withColumn("minute", F.minute(F.col("transaction_time")))

# Add a new column 'hour' extracted from 'transaction_time'
df = df.withColumn("second", F.second(F.col("transaction_time")))

# Show the updated DataFrame
df.show()

+--------------+----------------+-------------------+---------------+--------+---------------+----------+----------+------------------+--------------------+--------------------+-----------+----+-------+---------+----+------+------+
|transaction_id|transaction_date|   transaction_time|transaction_qty|store_id| store_location|product_id|unit_price|  product_category|        product_type|      product_detail|Total_sales|year|  month|      day|hour|minute|second|
+--------------+----------------+-------------------+---------------+--------+---------------+----------+----------+------------------+--------------------+--------------------+-----------+----+-------+---------+----+------+------+
|           662|      2023-01-02|2024-08-14 11:14:15|              2|       3|        Astoria|        32|       3.0|            Coffee|Gourmet brewed co...|         Ethiopia Rg|        6.0|2023|January|   Monday|  11|    14|    15|
|           729|      2023-01-02|2024-08-14 12:26:13|              2|   

# Visualization

In [0]:
df.createOrReplaceTempView('Coffee_data')

In [0]:
%sql
select * from Coffee_data limit 5;

transaction_id,transaction_date,transaction_time,transaction_qty,store_id,store_location,product_id,unit_price,product_category,product_type,product_detail,Total_sales,year,month,day,hour,minute,second
662,2023-01-02,2024-08-14T11:14:15.000+0000,2,3,Astoria,32,3.0,Coffee,Gourmet brewed coffee,Ethiopia Rg,6.0,2023,January,Monday,11,14,15
729,2023-01-02,2024-08-14T12:26:13.000+0000,2,5,Lower Manhattan,25,2.2,Coffee,Organic brewed coffee,Brazilian Sm,4.4,2023,January,Monday,12,26,13
838,2023-01-02,2024-08-14T14:21:55.000+0000,2,8,Hell's Kitchen,41,4.25,Coffee,Barista Espresso,Cappuccino Lg,8.5,2023,January,Monday,14,21,55
866,2023-01-02,2024-08-14T14:58:28.000+0000,2,8,Hell's Kitchen,46,2.5,Tea,Brewed Green tea,Serenity Green Tea Rg,5.0,2023,January,Monday,14,58,28
881,2023-01-02,2024-08-14T15:22:19.000+0000,2,8,Hell's Kitchen,28,2.0,Coffee,Gourmet brewed coffee,Columbian Medium Roast Sm,4.0,2023,January,Monday,15,22,19


In [0]:
distinct_values_list = [row.store_location for row in df.select("store_location").distinct().collect()]

In [0]:
%python
# Create the a dropdown widget
dbutils.widgets.dropdown(name='store_location', defaultValue='Astoria', choices=distinct_values_list , label='Dropdown')

In [0]:
%sql 
-- Total sales by product category
select product_category,Total_sales from Coffee_data
-- Highest sales is are Coffee then 2nd highest sales Tea and 3rd hightest bakery

product_category,Total_sales
Coffee,6.0
Coffee,4.4
Coffee,8.5
Tea,5.0
Coffee,4.0
Tea,6.2
Drinking Chocolate,9.0
Tea,3.0
Coffee,4.25
Bakery,3.0


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- Top 5 Total_sales by product detail
select product_type,sum(Total_sales) as Total_sales from Coffee_data group by product_type order by Total_sales desc limit 5;

product_type,Total_sales
Barista Espresso,91406.20000000013
Brewed Chai tea,77081.9500000005
Hot chocolate,72416.0
Gourmet brewed coffee,70034.59999999967
Brewed Black tea,47932.0


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- Lowest 5 Total_sales by product detail
select product_type,sum(Total_sales) as Total_sales from Coffee_data group by product_type order by Total_sales limit 5;

product_type,Total_sales
Green beans,1340.0
Green tea,1470.75
Organic Chocolate,1679.5999999999992
Sugar free syrup,2324.000000000004
Black tea,2711.8499999999985


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- product type vs Total Quantity sold
-- Top 10
select product_type,store_location,sum(transaction_qty) as `Total_qty` from Coffee_data group by product_type,store_location order by Total_qty desc limit 10;

product_type,store_location,Total_qty
Brewed Chai tea,Astoria,9306
Barista Espresso,Hell's Kitchen,9064
Gourmet brewed coffee,Astoria,8938
Brewed Chai tea,Hell's Kitchen,8755
Gourmet brewed coffee,Lower Manhattan,8563
Barista Espresso,Lower Manhattan,8534
Gourmet brewed coffee,Hell's Kitchen,8472
Brewed Chai tea,Lower Manhattan,8189
Barista Espresso,Astoria,7345
Hot chocolate,Astoria,6351


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- Total sales vs month
select sum(Total_sales),month from Coffee_data group by month ;

sum(Total_sales),month
76145.19000000006,February
81677.74000000008,January
98834.68000000021,March
156727.76000000024,May
118941.08000000034,April
166485.88000000018,June


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- Distribution of Total sales
select Total_sales from Coffee_data

Total_sales
6.0
4.4
8.5
5.0
4.0
6.2
9.0
3.0
4.25
3.0


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- transation date vs total sales
select sum(Total_sales) as `Total_sales`,transaction_date from Coffee_data group by transaction_date 

Total_sales,transaction_date
5615.100000000001,2023-06-22
5541.160000000001,2023-05-22
2823.5499999999997,2023-02-25
6026.09,2023-06-18
2762.4300000000003,2023-02-08
5781.860000000001,2023-06-23
3088.3299999999995,2023-03-12
3586.2000000000003,2023-03-24
2508.2,2023-01-01
4714.6,2023-05-03


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- sales vs product categories
select sum(Total_sales),store_location
 from Coffee_data group by store_location

sum(Total_sales),store_location
230057.24999999942,Lower Manhattan
236511.16999999972,Hell's Kitchen
232243.91000000038,Astoria


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
select sum(Total_sales) as `Total sales` from Coffee_data

Total sales
698812.3299999984


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
select transaction_qty, month from Coffee_data order by transaction_qty desc;

transaction_qty,month
8,January
8,April
8,January
8,June
8,April
8,May
8,June
8,June
8,June
8,May


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
select transaction_qty, day from Coffee_data;

transaction_qty,day
2,Monday
2,Monday
2,Monday
2,Monday
2,Monday
2,Tuesday
2,Tuesday
1,Wednesday
1,Wednesday
1,Wednesday


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
select Total_sales,hour from Coffee_data;

Total_sales,hour
6.0,11
4.4,12
8.5,14
5.0,14
4.0,15
6.2,14
9.0,15
3.0,10
4.25,13
3.0,14
